# Importing

In [ ]:
#%%capture

# Intel® Extension for Scikit-learn installation:
#
!pip install scikit-learn-intelex
!pip install metric-learn
import os
import warnings

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import mode
from tqdm import tqdm
from pathlib import Path



from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from math import factorial

# Mute warnings
warnings.filterwarnings("ignore")

Thanks @sy-tuan nguyen for the memory reducing routine

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
data_dir = Path('../input/tabular-playground-series-feb-2022')

df_train = pd.read_csv(data_dir / 'train.csv', index_col='row_id').pipe(reduce_mem_usage)
df_test  = pd.read_csv(data_dir / 'test.csv', index_col='row_id').pipe(reduce_mem_usage)
TARGET = df_train.columns.difference(df_test.columns)[0]
features = df_train.columns[df_train.columns != TARGET]

# Preprocessing
There are two preprocessing steps here. First I drop the duplicates, then I match the means of train and test to partially counteract drift in the populations. Then I standardize the data, which you should pretty much always do for KNN. I tried some other scalers and they gave worse results.
Because KNN takes so long to run when predicting, instead of performing cross-validation I split the training data into train and validation. The validation set is small because we have a lot of data, so we don't need a large validation dataset to get accurate results. The validation set still has 10000 examples

In [ ]:
df_train.drop_duplicates(keep='first', inplace=True)

In [ ]:
def pre_process(train, test):
    return train + (test.mean(axis=0) - train.mean(axis=0))
df_train[features] = pre_process(df_train[features], df_test[features])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
# Encoding categorical features
le = LabelEncoder()
scaler = StandardScaler()
X = df_train[features]
y = pd.DataFrame(le.fit_transform(df_train[TARGET]), columns=[TARGET])
X = scaler.fit_transform(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, random_state=42)

In [ ]:
X_test = df_test[features]
X_test = scaler.transform(X_test)

# Training
I tuned the number of neighbors and the distance metric.

In [ ]:
%%time
model = KNeighborsClassifier(n_neighbors=4, p=1)
model.fit(X_train, y_train)
model.score(X_valid, y_valid)

# Submission
Here I retrain the model on the full data

In [ ]:
model = KNeighborsClassifier(n_neighbors=4, p=1)
model.fit(X, y)
y_pred = model.predict(X_test)
y_pred = le.inverse_transform(y_pred)

In [ ]:

submission = pd.read_csv(data_dir / 'sample_submission.csv')
submission[TARGET] = y_pred
submission.to_csv('submission.csv', index=False)
submission